In [ ]:
# https://machinelearningmastery.com/multi-step-time-series-forecasting-long-short-term-memory-networks-python/

import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import math
import matplotlib.pyplot as plt

np.random.seed(1337)

# Creating functions

In [ ]:
# transform series into train and test sets for supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names

    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg


In [ ]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
    # extract raw values
    raw_values = series.values

    # transform data to be stationary
    diff_values = raw_values
    diff_values = diff_values.reshape(len(diff_values), 1)

    # rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_values)
    scaled_values = scaled_values.reshape(len(scaled_values), 1)

    # transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values

    # split into train and test sets
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [ ]:
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, 0:n_lag], train[:, n_lag:]

    X = X.reshape(X.shape[0], 1, X.shape[1])

    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss="mean_squared_error", optimizer="adam")

    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=2, shuffle=False)

    return model

In [ ]:
# make one forecast with an LSTM,
def forecast_lstm(model, X, n_batch):
    # reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, 1, len(X))

    # make forecast
    forecast = model.predict(X, batch_size=n_batch)

    # convert to array
    return [x for x in forecast[0, :]]

In [ ]:
# evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    print(f'Forecast x of {len(test)}:', end=" ")
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # store the forecast
        forecasts.append(forecast)

        # Printing current status in hundreds
        hundred = i % 100
        if hundred == 0:
            print(i, end=" ")

    return forecasts

In [ ]:
# inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):

        # create array from forecast
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))

        # invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]

        inverted.append(inv_scale)

    return inverted

# Fitting and predicting

In [ ]:
# load dataset
logreturns = "data/final.csv"
series = pd.read_csv(logreturns, usecols=["Exchange.Date", "logreturns"], header=0, index_col=0, squeeze=True)

# configure
n_lag = 63
n_seq = 63  #  number of periods forecast
test_share = 0.25
n_test = int(len(series) * test_share)
n_epochs = 5
n_batch = 1
n_neurons = 50

print("Preparing data...")
scaler, train, test = prepare_data(series, n_test, n_lag, n_seq)

print("Fitting model...")
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

print("Making forecasts...")
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)

print("\nInverting forecasts...")
forecasts = inverse_transform(series, forecasts, scaler)
print("Done!")

# Evaluating from t=1


## Creating dataframe for evaluation
In essence, creating a new DF combining training data (historic) and forecasts

In [ ]:
# Getting dataframe with Close as well and the creating a training df same size as used in the model
original_df = pd.read_csv("data/final.csv", usecols=["logreturns", "Close"])
train_df = original_df[:-n_test].copy()

# Assigning all rows in train df (before forecast) to closing value
# This is because this column cannot be empty (and we have no forecasts since it's training data)
train_df["forecast"] = train_df["Close"]

In [ ]:
# Transforming logreturns back to price
last_train = train_df["Close"].values[-1]
price_forecasts = np.exp(np.cumsum(forecasts[0]) + math.log(last_train))

In [ ]:
# Creating a separate dataframe only for forecasts (i.e. "outside train df")
forecast_df = pd.DataFrame(columns=["Close", "logreturns", "forecast"])
forecast_df["Close"] = original_df["Close"].values[-n_test : -n_test + n_seq]
forecast_df["logreturns"] = original_df["logreturns"].values[-n_test : -n_test + n_seq]
forecast_df["forecast"] = price_forecasts

In [ ]:
# Merging train and forecast dataframe
merged_df = train_df.append(forecast_df, ignore_index=True)

# Creating error, absolute error, actual price going up (True/False) and forecast going up (True/False)
merged_df["error"] = merged_df["forecast"] - merged_df["Close"]
merged_df["abs_error"] = np.abs(merged_df["forecast"] - merged_df["Close"])
merged_df["actual_up"] = merged_df["Close"].diff(1) > 0
merged_df["forecast_up"] = merged_df["forecast"].diff(1) > 0

# Formula for creating confusion value, used below
def confusion(actual, forecast):
    if actual and forecast:
        return "TP"

    if actual and not forecast:
        return "FN"

    if not actual and forecast:
        return "FP"

    if not actual and not forecast:
        return "TN"

    # Just common programming sense to return something, could have written "blabla"
    return False


# The lambda stuff applies the above function on every row of data
merged_df["confusion"] = merged_df.apply(lambda x: confusion(x["actual_up"], x["forecast_up"]), axis=1)

# Printing the tail of the data
merged_df.tail()

## Evaluating

In [ ]:
# New dataframe that only contains the number of periods to evaluate (1,3,5,21,63)
def new_df(n_periods):
    df = merged_df[len(train_df) : len(train_df) + n_periods]
    return df

In [ ]:
# Creating RMSE AND MAE
def evaluate(n_periods):
    df = new_df(n_periods)
    mape = ((df["abs_error"] / df["Close"]).sum() / n_periods) * 100
    rmse = math.sqrt(pow(df["error"].sum(), 2) / n_periods)
    print(f"{n_periods}, RMSE: {round(rmse, 3)}, MAPE: {round(mape, 3)}%")


evaluate(1)  # 1 day
evaluate(3)  # half a week
evaluate(5)  # week
evaluate(21)  # month
evaluate(63)  # quarter

In [ ]:
# Creating confusion matrix
def confusion_matrix(df):
    conf = pd.DataFrame(columns=["P", "N"], index=["P", "N"])
    conf.loc["P", "P"] = len(df[df["confusion"] == "TP"])
    conf.loc["P", "N"] = len(df[df["confusion"] == "FN"])
    conf.loc["N", "P"] = len(df[df["confusion"] == "FP"])
    conf.loc["N", "N"] = len(df[df["confusion"] == "TN"])
    return conf


confusion = confusion_matrix(new_df(63))
precision = confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[0, 1])
recall = confusion.iloc[0, 0] / (confusion.iloc[0, 0] + confusion.iloc[1, 0])
f_score = 2 * precision * recall / (precision + recall)

print(confusion)
print(f"precision: {int(precision*100)}%, recall: {int(recall*100)}%, f-score: {round(f_score, 3)}")

# Plotting

In [ ]:
plot_df = merged_df[-n_seq - (n_seq * 2) :]
plt.figure(figsize=(10, 5))
plt.plot(plot_df["forecast"], label="forecast")
plt.plot(plot_df["Close"], label="actual")
plt.legend()